In [2]:
from dataset import get_datasets

train_dataset, test_dataset = get_datasets()

In [3]:
import pandas as pd
import torch.nn as nn
from torchvision import models
from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
alexnet = models.alexnet(pretrained=True)
resnet18 = models.resnet18(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

last_layer_classifier_alexnet = nn.Sequential(
    nn.Linear(9216, len(df.columns) - 2), 
    nn.Sigmoid()
)
last_layer_classifier_resnet18 = nn.Sequential(
    nn.Linear(resnet18.fc.in_features, len(df.columns) - 2), 
    nn.Sigmoid()
)
last_layer_classifier_resnet50 = nn.Sequential(
    nn.Linear(resnet50.fc.in_features, len(df.columns) - 2), 
    nn.Sigmoid()
)


c:\Users\USER\anaconda3\envs\playtorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\anaconda3\envs\playtorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\USER\anaconda3\envs\playtorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18

In [7]:
import wandb

PROJECT_NAME = "STAT4012Project_SGD"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [5]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from torch import optim
from model import pretrainedModel
import time

config = {
    "batch_size": 32,
    "loss_fn": nn.BCELoss,
    "lr": 0.0001,
    "optimizer": optim.SGD,
    "epochs": 5,
}

model_alexnet_SGD = pretrainedModel(
    alexnet,
    last_layer_classifier_alexnet,
    len(df.columns) - 2,
    config["loss_fn"],
    config["optimizer"],
    lr=config["lr"],
)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=6, pin_memory=True)
validation_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=6, pin_memory=True)

run_name = "alexnetSGD_" + str(int(time.time()))
logger = WandbLogger(name=run_name, project=PROJECT_NAME, log_model="all")
trainer = L.Trainer(max_epochs=config["epochs"], logger=logger, accelerator="gpu", devices=1, default_root_dir="AlexNet")

trainer.fit(model_alexnet_SGD, train_loader, validation_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type            | Params
-----------------------------------------------------
0 | features         | Sequential      | 2.5 M 
1 | class_classifier | Sequential      | 175 K 
2 | loss_fn          | BCELoss         | 0     
3 | metrics          | MultilabelAUROC | 0     
-----------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.579    Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.59it/s]

c:\Users\USER\anaconda3\envs\playtorch\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 4: 100%|██████████| 13721/13721 [19:02<00:00, 12.01it/s, v_num=vryz, train_auroc=0.541, train_loss_step=0.253, val_auroc_step=0.275, val_loss_step=0.216, val_auroc_epoch=0.534, val_loss_epoch=0.230, train_loss_epoch=0.231]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 13721/13721 [19:03<00:00, 12.00it/s, v_num=vryz, train_auroc=0.541, train_loss_step=0.253, val_auroc_step=0.275, val_loss_step=0.216, val_auroc_epoch=0.534, val_loss_epoch=0.230, train_loss_epoch=0.231]


In [ ]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from torch import optim
from model import pretrainedModel
import time

config = {
    "batch_size": 32,
    "loss_fn": nn.BCELoss,
    "lr": 0.0001,
    "optimizer": optim.SGD,
    "epochs": 5,
}

model_resnet18_SGD = pretrainedModel(
    resnet18,
    last_layer_classifier_resnet18,
    len(df.columns) - 2,
    config["loss_fn"],
    config["optimizer"],
    lr=config["lr"],
)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True,num_workers=6, pin_memory=True)
validation_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False,num_workers=6, pin_memory=True)

run_name = "resnet18_" + str(int(time.time()))
logger = WandbLogger(name=run_name, project=PROJECT_NAME, log_model="all")
trainer = L.Trainer(max_epochs=config["epochs"], logger=logger, accelerator="gpu", devices=1)

print(run_name)
trainer.fit(model_resnet18_SGD, train_loader, validation_loader, ckpt_path = "STAT4012Project_SGD/nyg6vryz/checkpoints/epoch=1-step=27442.ckpt")

In [ ]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from torch import optim
import time


from model import pretrainedModel


config = {"batch_size": 32, 
          "loss_fn": nn.BCELoss, 
          "optimizer": optim.SGD, 
          "lr": 0.0001,
          "epochs": 5}

model_resnet50_SGD = pretrainedModel(
    resnet50,
    last_layer_classifier_resnet50,
    len(df.columns) - 2,
    config["loss_fn"],
    config["optimizer"],
    lr=config["lr"],
)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True,num_workers=6, pin_memory=True)
validation_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False,num_workers=6, pin_memory=True)

run_name = "resnet50_" + str(int(time.time()))
logger = WandbLogger(name=run_name, project=PROJECT_NAME, log_model="all")
trainer = L.Trainer(max_epochs=config["epochs"], logger=logger, accelerator="gpu", devices=1, default_root_dir="ResNet50")

print(run_name)
trainer.fit(model_resnet50_SGD, train_loader, validation_loader)